## Load/import packages

In [2]:
import json
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from os import listdir, mkdir
from os.path import splitext
from os.path import join
from skimage import io, color

%matplotlib inline

# Check if Tensorflow uses GPU
print(tf.config.experimental.list_physical_devices("GPU"))

# Limit GPU memory usage
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print()
print(f"Tensorflow Version: {tf.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"Matplotlib Version: {matplotlib.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Tensorflow Version: 2.4.0-rc0
Numpy Version: 1.19.2
OpenCV Version: 4.4.0
Matplotlib Version: 3.3.1
Keras Version: 2.4.0


# Load datasets

In [8]:
AW2_train_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets_per_class\train"
AW2_norm_train = image_dataset_from_directory(
    directory=AW2_train_dir,
    labels="inferred",
    label_mode="categorical",
    shuffle=False,
    color_mode="rgb",
    batch_size=1,
    image_size=(112, 112),
)

Found 922029 files belonging to 7 classes.


In [9]:
# Pull the first batch of 7 images
count = 0
for image, label in AW2_norm_train:
    if count == 1:
        break
    count += 1
    img = image
    lbl = label

In [10]:
image.shape

TensorShape([1, 112, 112, 3])

# Initiate Base CNN
For the feature extraction the pre-trained **VGG19** network will be used with the imagenet weights. Input shape is set to 112,112,3

In [17]:
from tensorflow.keras.applications.vgg19 import preprocess_input

base_VGG19 = tf.keras.applications.VGG19(
    include_top=False, weights="imagenet", input_shape=(112, 112, 3)
)

In [18]:
base_VGG19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0     

# Extract all Features

In [29]:
# Define ImageDataGenerator with precoessing function set to preprocess_input for vgg19 model
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    directory=AW2_train_dir,
    target_size=(112, 112),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
)

Found 922029 images belonging to 7 classes.


In [33]:
print(f'Amount of images in training set: {train_generator.n}')
print(f'Batch size of generator: {train_generator.batch_size}')

Amount of images in training set: 922029
Batch size of generator: 32


In [ ]:
# Reset batch index to 0 again for the train_generator
train_generator.reset()

In [ ]:
pred = base_VGG19.predict(train_generator,)

In [24]:
def build_feature_extractor():
    model = Sequential(name="CNN-Feature extaction")
    model.add(base_VGG19)
    base_VGG19.trainable = False
    model.add(layers.Flatten(name="Flatten"))
    model.compile()
    return model


FE_model = build_feature_extractor()

In [25]:
FE_model.summary()

Model: "CNN-Feature extaction"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
Flatten (Flatten)            (None, 4608)              0         
Total params: 20,024,384
Trainable params: 0
Non-trainable params: 20,024,384
_________________________________________________________________
